# Document Similarity

# Rekomendasi Buku dengan Kemiripan Dokumen

Rekomendasi buku adalah aplikasi yang serupa dengan rekomendasi film, tetapi dalam konteks rekomendasi buku, kita ingin merekomendasikan buku kepada pembaca berdasarkan minat mereka. Berikut adalah tiga pendekatan yang serupa dengan rekomendasi buku:

Simple Rule-based Recommenders :
- Rekomendasi berdasarkan popularitas buku: Menampilkan buku-buku populer yang banyak dibaca atau dinilai tinggi oleh pembaca lain.
- Rekomendasi berdasarkan genre: Memberikan buku-buku dalam genre yang sesuai dengan minat pembaca. Misalnya, jika seseorang suka fiksi ilmiah, rekomendasikan buku-buku dalam genre tersebut.

Content-based Recommenders:
- Rekomendasi berdasarkan atribut buku: Menganalisis atribut konten buku seperti deskripsi, genre, penulis, dan kata kunci. Selain itu, dapat merekomendasikan buku dengan atribut yang serupa dengan buku yang telah disukai oleh pembaca.
- Analisis sentimen ulasan buku: Menganalisis sentimen ulasan pembaca untuk buku-buku dan merekomendasikan buku dengan sentimen positif yang mirip dengan buku yang disukai.

Collaborative Filtering Recommenders:
- Menganalisis preferensi pembaca: Menganalisis preferensi pembaca berdasarkan buku yang telah mereka baca atau dinilai. Jika dua pembaca memiliki preferensi yang serupa, kita dapat merekomendasikan buku yang satu telah membaca kepada yang lain.
- Filtering berbasis peringkat: Menggunakan peringkat yang diberikan oleh pembaca untuk buku-buku tertentu. Jika dua pembaca memberikan peringkat yang mirip untuk buku yang sama, sistem dapat merekomendasikan buku tersebut kepada keduanya.


In [1]:
!pip install textsearch
!pip install contractions
!pip install nltk
import nltk
nltk.download('punkt')
nltk.download('stopwords')


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import pathlib
pathlib.Path().resolve()

WindowsPath('C:/Users/User/Projek-UAS')

# load dan view data

In [3]:
import pandas as pd

df = pd.read_csv("goodreads_data.csv")
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   10000 non-null  int64  
 1   Book         10000 non-null  object 
 2   Author       10000 non-null  object 
 3   Description  9923 non-null   object 
 4   Genres       10000 non-null  object 
 5   Avg_Rating   10000 non-null  float64
 6   Num_Ratings  10000 non-null  object 
 7   URL          10000 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 625.1+ KB


,Unnamed: 0,Book,Author,Description,Genres,Avg_Rating,Num_Ratings,URL
0,0,To Kill a Mockingbird,Harper Lee,The unforgettable novel of a childhood in a sl...,"['Classics', 'Fiction', 'Historical Fiction', ...",4.27,"5,691,311",https://www.goodreads.com/book/show/2657.To_Ki...
1,1,Harry Potter and the Philosopher’s Stone (Harr...,J.K. Rowling,Harry Potter thinks he is an ordinary boy - un...,"['Fantasy', 'Fiction', 'Young Adult', 'Magic',...",4.47,"9,278,135",https://www.goodreads.com/book/show/72193.Harr...
2,2,Pride and Prejudice,Jane Austen,"Since its immediate success in 1813, Pride and...","['Classics', 'Fiction', 'Romance', 'Historical...",4.28,"3,944,155",https://www.goodreads.com/book/show/1885.Pride...
3,3,The Diary of a Young Girl,Anne Frank,Discovered in the attic in which she spent the...,"['Classics', 'Nonfiction', 'History', 'Biograp...",4.18,"3,488,438",https://www.goodreads.com/book/show/48855.The_...
4,4,Animal Farm,George Orwell,Librarian's note: There is an Alternate Cover ...,"['Classics', 'Fiction', 'Dystopia', 'Fantasy',...",3.98,"3,575,172",https://www.goodreads.com/book/show/170448.Ani...


In [4]:
df = df[['Book', 'Author', 'Genres', 'Num_Ratings']]
df.Author.fillna('', inplace=True)
df['description'] = df['Author'].map(str) + ' ' + df['Genres']
df.dropna(inplace=True)
df = df.sort_values(by=['Num_Ratings'], ascending=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 1468 to 3747
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Book         10000 non-null  object
 1   Author       10000 non-null  object
 2   Genres       10000 non-null  object
 3   Num_Ratings  10000 non-null  object
 4   description  10000 non-null  object
dtypes: object(5)
memory usage: 468.8+ KB


In [5]:
df.head()

,Book,Author,Genres,Num_Ratings,description
1468,"Hometown Girl After All (Hometown, #2)",Kirsten Fullmer,"['Contemporary', 'Young Adult', 'New Adult', '...",999,"Kirsten Fullmer ['Contemporary', 'Young Adult'..."
3033,"Hometown Girl After All (Hometown, #2)",Kirsten Fullmer,"['Contemporary', 'Young Adult', 'New Adult', '...",999,"Kirsten Fullmer ['Contemporary', 'Young Adult'..."
4735,"Belonging (Temptation, #2)",Karen Ann Hopkins,"['Young Adult', 'Romance', 'Amish', 'Contempor...",998,"Karen Ann Hopkins ['Young Adult', 'Romance', '..."
7411,تشي,أحمد خالد توفيق,"['Fiction', 'Novels', 'Fantasy']",998,"أحمد خالد توفيق ['Fiction', 'Novels', 'Fantasy']"
2796,Living The Best Day Ever,Hendri Coetzee,"['Nonfiction', 'Adventure']",997,"Hendri Coetzee ['Nonfiction', 'Adventure']"


In [6]:
df.iloc[4799].description

"Georgette Heyer ['Romance', 'Historical Fiction', 'Historical Romance', 'Historical', 'Fiction', 'Regency', 'Classics']"

# Bangun Sistem Rekomendasi Buku

Tahapan
- Pre Processing
- Feature Engineering
- Komputasi Doc Similarity
- Proses Retrieve
- proses rekomendasi film


## Kemiripan Dokumen / document similarity

Ada berbagai cara untuk menghitung kesamaan antara dua item dokumen. Salah satu ukuran yang paling banyak digunakan adalah __cosine similarity__ .

### Cosine Similarity

Cosine Similarity digunakan untuk menghitung skor numerik untuk menunjukkan kesamaan antara dua dokumen teks. Secara matematis, ini didefinisikan sebagai berikut:


In [7]:
# Select the top 30,000 rows
book_500_df = df.head(500)

# Check the information of the new DataFrame
book_500_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500 entries, 1468 to 5083
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Book         500 non-null    object
 1   Author       500 non-null    object
 2   Genres       500 non-null    object
 3   Num_Ratings  500 non-null    object
 4   description  500 non-null    object
dtypes: object(5)
memory usage: 23.4+ KB


In [8]:
import nltk
import re
import numpy as np
import contractions

stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    doc = contractions.fix(doc)
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    #filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

norm_corpus = normalize_corpus(list(book_500_df['description']))
len(norm_corpus)

500

## Extrak TF-IDF

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(ngram_range=(1, 2), min_df=2)
tfidf_matrix = tf.fit_transform(norm_corpus)
tfidf_matrix.shape

(500, 838)

## Compute Pairwise Document Similarity

In [10]:
from sklearn.metrics.pairwise import cosine_similarity

doc_sim = cosine_similarity(tfidf_matrix)
doc_sim_df = pd.DataFrame(doc_sim)
doc_sim_df.head()

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,1.000000,1.000000,0.212011,0.013984,0.0,0.099264,0.010545,0.049507,0.061947,0.073470,...,0.0,0.000000,0.008162,0.0,0.000000,0.000000,0.000000,0.042473,0.0,0.000000
1,1.000000,1.000000,0.212011,0.013984,0.0,0.099264,0.010545,0.049507,0.061947,0.073470,...,0.0,0.000000,0.008162,0.0,0.000000,0.000000,0.000000,0.042473,0.0,0.000000
2,0.212011,0.212011,1.000000,0.017028,0.0,0.145530,0.012841,0.060284,0.075431,0.113254,...,0.0,0.000000,0.009939,0.0,0.000000,0.000000,0.000000,0.051718,0.0,0.000000
3,0.013984,0.013984,0.017028,1.000000,0.0,0.075481,0.117671,0.080822,0.225742,0.056965,...,0.0,0.000000,0.029744,0.0,0.000000,0.085166,0.000000,0.154776,0.0,0.000000
4,0.000000,0.000000,0.000000,0.000000,1.0,0.137711,0.000000,0.000000,0.000000,0.132871,...,0.0,0.119752,0.000000,0.0,0.088458,0.000000,0.077747,0.000000,0.0,0.091851


In [11]:
doc_sim_df.to_csv("similarity.csv")

In [12]:
book_500_df['Book'] = book_500_df['Book'].astype(str)

C:\Users\User\AppData\Local\Temp\ipykernel_8288\1746086328.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  book_500_df['Book'] = book_500_df['Book'].astype(str)


In [13]:
book_500_df['Book'] = book_500_df['Book'].str.lower()

C:\Users\User\AppData\Local\Temp\ipykernel_8288\1515595456.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  book_500_df['Book'] = book_500_df['Book'].str.lower()


In [14]:
book_500_df.to_csv("book_500.csv")

## mendapatkan judul buku

In [15]:
book_list = book_500_df['Book'].values
book_list, book_list.shape

(array(['hometown girl after all (hometown, #2)',
        'hometown girl after all (hometown, #2)',
        'belonging (temptation, #2)', 'تشي', 'living the best day ever',
        'allegiant (divergent, #3)', 'the raj quartet',
        'outlander (outlander, #1)', 'زندگی مه آلود پریا',
        'the demigod files', "shadow's edge (night angel, #2)",
        'those who save us', 'beezus and ramona (ramona, #1)',
        'the foundation trilogy (foundation, #1-3)', 'pygmalion',
        'braving the wilderness: the quest for true belonging and the courage to stand alone',
        'dreamfever (fever, #4)', 'haunted', 'we were the mulvaneys',
        'ham on rye', 'glass houses (the morganville vampires, #1)',
        'edge of eternity (the century trilogy, #3)',
        'la santa biblia: antiguo y nuevo testamentos',
        'teatro de gil vicente', 'from the falls (the falls trilogy, #2)',
        'blackamoores: africans in tudor england, their presence, status and origins',
        'barb

In [16]:
book_list

array(['hometown girl after all (hometown, #2)',
       'hometown girl after all (hometown, #2)',
       'belonging (temptation, #2)', 'تشي', 'living the best day ever',
       'allegiant (divergent, #3)', 'the raj quartet',
       'outlander (outlander, #1)', 'زندگی مه آلود پریا',
       'the demigod files', "shadow's edge (night angel, #2)",
       'those who save us', 'beezus and ramona (ramona, #1)',
       'the foundation trilogy (foundation, #1-3)', 'pygmalion',
       'braving the wilderness: the quest for true belonging and the courage to stand alone',
       'dreamfever (fever, #4)', 'haunted', 'we were the mulvaneys',
       'ham on rye', 'glass houses (the morganville vampires, #1)',
       'edge of eternity (the century trilogy, #3)',
       'la santa biblia: antiguo y nuevo testamentos',
       'teatro de gil vicente', 'from the falls (the falls trilogy, #2)',
       'blackamoores: africans in tudor england, their presence, status and origins',
       'barbarians: how the 

# Temukan Buku Serupa Teratas untuk Contoh Buku
Mari ambil Hometown Girl After All (Hometown, #2) buku yang memiliki ratung paling populer dari kerangka data di atas dan coba temukan buku paling mirip yang dapat direkomendasikan

## ambil book ID

In [17]:
book_idx = np.where(book_list == 'allegiant (divergent, #3)')[0][0]
book_idx

5

## ambil similarities

In [18]:
book_similarities = doc_sim_df.iloc[book_idx].values
book_similarities

array([0.09926365, 0.09926365, 0.14552983, 0.0754805 , 0.1377106 ,
       1.        , 0.02479515, 0.14236065, 0.14565633, 0.10410666,
       0.1026089 , 0.02081366, 0.09240305, 0.17372744, 0.01151526,
       0.        , 0.08142235, 0.02744961, 0.06102176, 0.01302015,
       0.11154042, 0.04101712, 0.        , 0.01360738, 0.27940932,
       0.        , 0.        , 0.01962789, 0.02463231, 0.0289077 ,
       0.19494433, 0.08706456, 0.01194424, 0.19768202, 0.01643926,
       0.14713608, 0.0277715 , 0.0801032 , 0.12971361, 0.        ,
       0.11845895, 0.01292929, 0.13427891, 0.02783909, 0.        ,
       0.        , 0.        , 0.04190195, 0.1109738 , 0.32539733,
       0.        , 0.0958169 , 0.02566372, 0.        , 0.01259358,
       0.0411709 , 0.27024735, 0.05074009, 0.28485203, 0.08898183,
       0.04873479, 0.        , 0.03994188, 0.01863559, 0.01792181,
       0.026311  , 0.        , 0.        , 0.        , 0.1002726 ,
       0.12465671, 0.01146324, 0.0335684 , 0.11441   , 0.07712

## Get top 5 similar book IDs

In [19]:
similar_book_idxs = np.argsort(-book_similarities)[1:6]
similar_book_idxs

array([108,  78, 478, 209, 136], dtype=int64)

## Get top 5 similar books

In [20]:
similar_books = book_list[similar_book_idxs]
similar_books

array(['free four: tobias tells the divergent knife-throwing scene (divergent, #1.5)',
       'son (the giver, #4)',
       'the house of the scorpion (matteo alacran, #1)',
       'the ask and the answer (chaos walking, #2)',
       'revealing eden (save the pearls, #1)'], dtype=object)

## Buat fungsi rekomendasi buku untuk merekomendasikan 5 buku serupa teratas untuk buku apa pun

In [21]:
def book_recommender(book_title, books=book_list, doc_sims=doc_sim_df):
    # find book id
    book_idx = np.where(books == book_title)[0][0]
    # get book similarities
    book_similarities = doc_sims.iloc[book_idx].values
    # get top 5 similar book IDs
    similar_book_idxs = np.argsort(-book_similarities)[1:6]
    # get top 5 books
    similar_books = books[similar_book_idxs]
    # return the top 5 books
    return similar_books

In [22]:
book_500_df.head(30)

,Book,Author,Genres,Num_Ratings,description
1468,"hometown girl after all (hometown, #2)",Kirsten Fullmer,"['Contemporary', 'Young Adult', 'New Adult', '...",999,"Kirsten Fullmer ['Contemporary', 'Young Adult'..."
3033,"hometown girl after all (hometown, #2)",Kirsten Fullmer,"['Contemporary', 'Young Adult', 'New Adult', '...",999,"Kirsten Fullmer ['Contemporary', 'Young Adult'..."
4735,"belonging (temptation, #2)",Karen Ann Hopkins,"['Young Adult', 'Romance', 'Amish', 'Contempor...",998,"Karen Ann Hopkins ['Young Adult', 'Romance', '..."
7411,تشي,أحمد خالد توفيق,"['Fiction', 'Novels', 'Fantasy']",998,"أحمد خالد توفيق ['Fiction', 'Novels', 'Fantasy']"
2796,living the best day ever,Hendri Coetzee,"['Nonfiction', 'Adventure']",997,"Hendri Coetzee ['Nonfiction', 'Adventure']"
922,"allegiant (divergent, #3)",Veronica Roth,"['Young Adult', 'Dystopia', 'Fiction', 'Fantas...","995,102","Veronica Roth ['Young Adult', 'Dystopia', 'Fic..."
6951,the raj quartet,Paul Scott,"['Fiction', 'Historical Fiction', 'India', 'Hi...",995,"Paul Scott ['Fiction', 'Historical Fiction', '..."
273,"outlander (outlander, #1)",Diana Gabaldon,"['Historical Fiction', 'Romance', 'Fantasy', '...","990,607","Diana Gabaldon ['Historical Fiction', 'Romance..."
247,زندگی مه آلود پریا,سید مرتضی مصطفوی,['Fiction'],990,سید مرتضی مصطفوی ['Fiction']
8712,the demigod files,Rick Riordan,"['Fantasy', 'Mythology', 'Young Adult', 'Middl...","99,875","Rick Riordan ['Fantasy', 'Mythology', 'Young A..."


In [23]:
popular_books = ['mid ocean']

In [24]:
for Book in popular_books:
    print('Books:', Book )
    print('Top 5 recommended Books:', book_recommender(book_title=Book, books=book_list, doc_sims=doc_sim_df))
    print()

Books: mid ocean
Top 5 recommended Books: ['the broken girls'
 'tinker, tailor, soldier, spy (george smiley, #5; karla trilogy, #1)'
 'the girl who played with fire (millennium, #2)' 'pink slips'
 'if shadows could tell: an aurora steller mystery novel']



In [25]:
book_recommender('mid ocean',book_list, doc_sim_df)

array(['the broken girls',
       'tinker, tailor, soldier, spy (george smiley, #5; karla trilogy, #1)',
       'the girl who played with fire (millennium, #2)', 'pink slips',
       'if shadows could tell: an aurora steller mystery novel'],
      dtype=object)